# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [42]:
import logging
import os
import csv
import json
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.runconfig import RunConfiguration
from azureml.core import ScriptRunConfig, Datastore, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.pipeline.core import Pipeline, PipelineData, PipelineRun, TrainingOutput#, StepRun, PortDataReference
from azureml.train.estimator import Estimator

from azureml.train.hyperdrive import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import quniform, choice
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep

import xgboost as xgboost

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


In [10]:
ws = Workspace.from_config()
experiment_name = 'Custom_Health_Insurance_CS'

exp = Experiment(ws, experiment_name)
def_blob_store = Datastore(ws, "workspaceblobstore")

## Attach computetarget

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "MS-GPUcompute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6',# for GPU, use "STANDARD_NC6"
                                                           vm_priority = 'lowpriority', # optional
                                                           max_nodes=8)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 4)

Found existing cluster, use it.
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Health insurance cross sell"

#dataset = Dataset.get_by_name(workspace, name='Health insurance cross sell')
#dataset.to_pandas_dataframe()

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        dl_data = 'https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction/download'
        dataset = Dataset.Tabular.from_delimited_files(dl_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description="https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction")

#data = dataset.to_pandas_dataframe()

In [32]:
#FOR REMOTE USE
aml_run_config = RunConfiguration()
aml_run_config.target = compute_target
aml_run_config.environment.docker.enabled = True
aml_run_config.environment.docker.base_image = "mcr.microsoft.com/azureml/base:latest"

aml_run_config.environment.python.user_managed_dependencies = True

aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas','scikit-learn','numpy', 'matplotlib', 'seaborn', 'imblearn'],
    pip_packages=['joblib','azureml-sdk','phik', 'xgboost'],
    pin_sdk_version=False
)



In [43]:
env = Environment.from_conda_specification("xgboost", "environment.yml")

src = ScriptRunConfig(
                    source_directory='.',
                    script='train.py',
                    compute_target=compute_target,
                    environment=env,
)

In [25]:
if "models" not in os.listdir():
    os.mkdir("./models")
    
raw_data = dataset.as_named_input('raw_data')
#raw_data = DataReference(datastore=def_blob_store, data_reference_name="raw_data", path_on_datastore="20newsgroups/20news.pkl")
train_data = PipelineData("train_data", datastore=def_blob_store).as_dataset()
test_data = PipelineData("test_data", datastore=def_blob_store).as_dataset()
scaler_file = PipelineData("scaler_file", datastore=def_blob_store)
#model_file = PipelineData("model_file", datastore=def_blob_store)


metrics_output_name = 'metrics_output'
metrics_data = PipelineData(name='metrics_data',
                            datastore=def_blob_store,
                            pipeline_output_name=metrics_output_name,
                            training_output=TrainingOutput("Metrics"))

model_output_name = 'model_output'
saved_model = PipelineData(name='saved_model',
                            datastore=def_blob_store,
                            pipeline_output_name=model_output_name,
                            training_output=TrainingOutput("Model",
                                                           model_file="models/best_model.pb"))

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [60]:
#Create the different params that you will be using during training
ps = BayesianParameterSampling({
    '--learning_rate': choice(0.1, 0.05),
    '--n_estimators': quniform(600,2000,200),
    '--max_depth': quniform(8,20,2),
    '--subsample': choice(0.8,0.9),
    '--colsample_bytree': choice(0.8,0.9)
})


# Create a Xgboost estimator for use with train.py
#est = Estimator(source_directory='.', entry_script='train.py', compute_target=compute_target, use_gpu=True, conda_packages=[], pip_packages=[])

hyperdrive_run_config = HyperDriveConfig(
                                         hyperparameter_sampling=ps, 
                                         primary_metric_name='Accuracy',
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=100,
                                         max_concurrent_runs=4,
                                        #run_config = aml_run_config,
                                        policy=None,
                                        estimator=src
                                        )

In [58]:
source_directory="."
prep_step = PythonScriptStep(name="prep_step",
                        script_name="./prep.py",
                        arguments=["--data", raw_data,"--train", train_data, "--test", test_data, "--scaler", scaler_file],
                        inputs=[raw_data],
                        outputs=[train_data,test_data, scaler_file],
                        compute_target=compute_target,
                        runconfig=aml_run_config,
                        source_directory=source_directory,
                        allow_reuse=True)

In [61]:
hd_step = HyperDriveStep(
                        name="hd_step",
                        hyperdrive_config=hyperdrive_run_config,
                        estimator_entry_script_arguments=["--train", train_data, "--test", test_data, "--model", saved_model],
                        inputs=[train_data,test_data],
                        outputs=[saved_model])

In [56]:
conda_dep = CondaDependencies()
conda_dep.add_pip_package("azureml-sdk")

rcfg = RunConfiguration(conda_dependencies=conda_dep)

register_model_step = PythonScriptStep(script_name='register_model.py',
                                       name="register_model_step01",
                                       inputs=[saved_model],
                                       compute_target=compute_target,
                                       arguments=["--model", saved_model],
                                       allow_reuse=True,
                                       runconfig=rcfg)

register_model_step.run_after(hd_step)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [62]:
pipeline1 = Pipeline(workspace=ws, steps=[prep_step, hd_step, register_model_step])
pipeline1.validate()
pipeline_run = Experiment(ws, 'health_insuarance').submit(pipeline1, regenerate_outputs=False)
RunDetails(pipeline_run).show()

AttributeError: 'ScriptRunConfig' object has no attribute '_get_script_run_config'

In [ ]:
pipeline_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

In [ ]:
with open(metrics_output._path_on_datastore) as f:  
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service